In [1]:
import requests
FSS_API_KEY = "af7496af43c43215e81251c8d2497a23"
response = requests.get(f"http://finlife.fss.or.kr/finlifeapi/mortgageLoanProductsSearch.json?auth={FSS_API_KEY}&topFinGrpNo=050000&pageNo=1")

In [2]:
import json
with open("fss_test.json", "w") as json_file:
    json.dump(response.json(), json_file)

In [8]:
dict(response.json())['result']['baseList'][0]

{'dcls_month': '202410',
 'fin_co_no': '0010593',
 'fin_prdt_cd': '302301',
 'kor_co_nm': '한화생명보험주식회사',
 'fin_prdt_nm': '홈드림모기지론',
 'join_way': '영업점,모집인',
 'loan_inci_expn': '인지세(50%), 주택채권매입비용',
 'erly_rpay_fee': '기본형(36개월이내 원금상환시 상환금액의 1.2% * 잔존일수 / 36개월) 외 3개 방식',
 'dly_rate': '대출만기 경과 건을 포함하여 연체기간에 상관없이 정상금리 +3%를 적용 (단, 최고 19%)',
 'loan_lmt': '감정가의 최고 70%까지 (담보물소재지, 대출금액, 고객신용, 소득 등에 따라 차등적용)',
 'dcls_strt_day': '20220819',
 'dcls_end_day': None,
 'fin_co_subm_day': '202410181400'}

In [ ]:
import os
from langchain_community.document_loaders import TextLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# OpenAI API 키 설정
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"
from dotenv import load_dotenv
load_dotenv()


def optimize_langchain_process():
    # 1. 문서 로드 및 분할
    # TextLoader를 사용하여 금융 기사 파일을 로드합니다.
    # 이는 금융 기사 파일을 읽어와서 문서를 생성합니다.
    # TextLoader() 참고: https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/
    loader = JSONLoader("./fss_test.json", jq_schema='.result.baseList[]', text_content=False)
    documents = loader.load()

    # RecursiveCharacterTextSplitter를 사용하여 문서를 작은 청크로 나눕니다.
    # chunk_size=500은 각 청크가 최대 500자의 텍스트를 포함하도록 설정하여
    # 메모리 사용을 최적화하고 처리 속도를 높입니다.
    # chunk_overlap=50은 청크 간 50자의 중첩을 허용하여 문맥을 유지하고
    # 중요한 정보를 잃지 않도록 합니다.
    # 과제1과 비교하여 더 작은 chunk_size와 chunk_overlap을 사용한 이유는 다음과 같습니다:
    # 1. 최적화 목적: 더 세밀한 텍스트 분할을 시도하여 최적화를 목표로 합니다.
    # 2. 검색 정확도 향상: 더 작은 텍스트 단위로 분할하여 검색의 정확도를 높입니다.
    # RecursiveCharacterTextSplitter() 참고: https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = text_splitter.split_documents(documents)

    # 2. 벡터 저장소 생성
    # OpenAIEmbeddings를 사용하여 문서의 텍스트를 벡터로 변환합니다.
    # 이는 문서를 벡터로 변환하여 유사도 검색을 수행할 수 있도록 합니다.
    # OpenAIEmbeddings() 참고: https://python.langchain.com/docs/integrations/text_embedding/openai/
    embeddings = OpenAIEmbeddings()

    # Chroma.from_documents를 사용하여 임베딩된 벡터를 저장소에 저장합니다.
    # 이는 문서의 벡터를 저장소에 저장하여 유사도 검색을 수행할 수 있도록 합니다.
    # Chroma 클래스는 벡터 저장소를 생성하고 관리하는 역할을 합니다.
    # from_documents() 메서드는 주어진 문서와 임베딩을 사용하여 벡터 저장소를 초기화합니다.
    # documents 매개변수는 임베딩할 텍스트 데이터(splits)를 제공하며, embedding 매개변수는 텍스트를 벡터로 변환하는 임베딩 모델(embeddings)을 지정합니다.
    # Chroma 클래스 참고: https://python.langchain.com/docs/integrations/vectorstores/chroma/
    # vector store, from_documents() 참고: https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

    # 3. 메모리 설정
    # TODO: ConversationBufferMemory()를 사용하여 대화 기록을 저장하는 메모리를 생성하세요.
    # ConversationBufferMemory를 사용하여 대화 기록을 저장하는 메모리를 생성합니다.
    # memory_key="chat_history"는 대화 기록이 "chat_history"라는 키로 저장됨을 나타냅니다.
    # return_messages=True는 메모리가 전체 대화 기록을 메시지 목록으로 반환하도록 설정합니다.
    # ConversationBufferMemory() 참고: https://python.langchain.com/v0.1/docs/modules/memory/types/buffer/
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    # 4. 프롬프트 템플릿 설정
    # PromptTemplate을 사용하여 프롬프트 템플릿을 생성합니다.
    prompt_template = """당신은 금융 전문가입니다. 주어진 컨텍스트를 바탕으로 다음 질문에 대해 간결하고 정확하게 답변해주세요:

    컨텍스트: {context}

    질문: {question}

    가능한 한 객관적이고 사실에 기반한 답변을 제공해주세요.

    모든 답변은 context의 사실을 기반으로 해야 합니다.

    
    컨텍스트를 활용하여 다음과 같이 질문에 답변할 수 있어야 합니다.
    ex) 

    - 사용자: 주택가격이 3억이고 1억 정도를 대출받고 싶습니다. 10년 만기로 생각하고 있습니다.

    - 당신: 부산은행의 "BNK357금리안심모기지론"을 추천드립니다. 해당 상품은 최저 금리가 3.02에 천월 평균 금리가 3.41%로 낮은 축에 속합니다. 말씀하신 내용으로 계산한 월 평균 상환액은 976,601원입니다.

    - 사용자: 이 상품의 상환액을 계산해주세요

    - 당신: 월평균 상환액은 다음과 같이 계산할 수 있습니다.

    이자율 3.41%와 상환 기간 20년을 사용하여 1억원(100,000,000원)의 월 평균 상환액을 계산해보겠습니다.

    M=P×r(1+r)^n/(1+r)^n−1

    M: 월 상환액

    P: 대출 원금 (100,000,000원)

    r: 월 이자율 (연이율 3.41%를 12로 나눠서 월 이자율로 계산)

    n: 총 상환 기간의 월 수 (20년이므로 240개월)

    주어진 조건에 따라 계산된 월 평균 상환액은 약 575,346원입니다. 이 금액을 20년 동안 매월 상환하게 됩니다. ​
    """
    # TODO: PromptTemplate()을 사용하여 프롬프트 템플릿을 생성하세요.
    # PromptTemplate을 사용하여 프롬프트 템플릿을 생성합니다.
    # template 매개변수는 프롬프트의 형식(prompt_template)을 지정합니다.
    # input_variables 매개변수는 프롬프트에서 사용할 변수 목록(["context", "question"])을 지정합니다.
    # PromptTemplate() 참고: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html
    PROMPT = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

    # 5. 대화형 검색 체인 생성
    # ConversationalRetrievalChain.from_llm을 사용하여 대화형 검색 체인을 생성합니다.
    # llm 매개변수는 ChatOpenAI를 사용하여 설정하고, retriever 매개변수는 vectorstore.as_retriever()를 사용하여 설정합니다.
    # memory 매개변수는 대화 기록을 저장하는 메모리를 지정하고,
    # combine_docs_chain_kwargs 매개변수는 프롬프트 템플릿을 지정합니다.
    # ConversationalRetrievalChain 참고: https://python.langchain.com/api_reference/langchain/chains/langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain.html
    # from_llm() 메서드 참고: https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html
    chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(temperature=0, model_name="gpt-4o-mini"),
        retriever=vectorstore.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": PROMPT}
    )

    return chain

# 메인 실행 부분
if __name__ == "__main__":
    chain = optimize_langchain_process()

    questions = [
        # "청년에게 좋은 담보대출은 무엇인가요?",
        # "저는 월 500만원 정도를 벌고 있습니다. 이에 적절한 상품이 있나요?",
        # "이전 질문들의 내용을 고려하여, 추천해준 상품의 다양한 정보들을 계산해주세요."
        "청년에게 좋은 담보대출은 무엇인가요?",
        "저는 월 500만원 정도를 벌고 있습니다. 주택이 4억원이고 2억 정도를 대출받고 싶을 때 이에 적절한 상품이 있나요?, 상환 기간은 30년으로 생각하고 있어요",
        "그럼 월 상환액이 어느정도 되나요?"
    ]

    for i, question in enumerate(questions, 1):
        result = chain({"question": question})
        print(f"\n질문 {i}: {question}")
        print(f"답변 {i}: {result['answer']}")
        print("-" * 50)


C:\Users\Kwon\AppData\Local\Temp\ipykernel_6956\3080248925.py:57: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\Kwon\AppData\Local\Temp\ipykernel_6956\3080248925.py:130: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"question": question})



질문 1: 청년에게 좋은 담보대출은 무엇인가요?
답변 1: 청년에게 적합한 담보대출 상품으로는 LTV(Loan To Value) 비율이 30%에서 70% 사이인 대출 상품을 추천드립니다. 이 비율은 담보로 제공하는 자산의 가치에 따라 대출 가능 금액이 결정되므로, 청년들이 상대적으로 적은 자산으로도 대출을 받을 수 있는 장점이 있습니다.

특히, 대출 금리가 0.8%에서 1.5% 사이로 설정된 상품이 있으며, 대출 시 적용되는 일일 이자율은 기본 금리에 3%가 추가된 형태입니다. 예를 들어, 대출 금리가 3%일 경우, 최종 이자율은 6%가 됩니다. 

대출 한도는 LTV 30%~70%로 설정되어 있어, 청년들이 주택 구매를 위해 필요한 자금을 마련하는 데 유리합니다. 

따라서, 청년들이 대출을 고려할 때는 LTV 비율과 이자율을 잘 비교하여 자신에게 맞는 상품을 선택하는 것이 중요합니다.
--------------------------------------------------

질문 2: 저는 월 500만원 정도를 벌고 있습니다. 주택이 4억원이고 2억 정도를 대출받고 싶을 때 이에 적절한 상품이 있나요?, 상환 기간은 30년으로 생각하고 있어요
답변 2: 주택 가격이 4억원이고 2억원을 대출받고자 하신다면, LTV(Loan To Value) 비율이 50%로, 이는 30%~70% 범위 내에 해당하므로 대출이 가능합니다. 

상환 기간을 30년으로 설정하셨을 때, 대출 상품을 선택할 수 있습니다. 제공된 컨텍스트에 따르면, 대출 금리는 0.8%에서 1.5% 사이입니다. 이를 바탕으로 월 평균 상환액을 계산해보겠습니다.

1. **대출 원금(P)**: 200,000,000원
2. **이자율(r)**: 연 1.5%를 기준으로 계산 (최대 금리 사용)
   - 월 이자율 = 1.5% / 12 = 0.125% = 0.00125
3. **상환 기간(n)**: 30년 = 360개월

월 상환액(M)은 다음과 같이 계산됩니다:

\[ 
M = P \times \frac

In [13]:
chain = optimize_langchain_process()

questions = [
    "청년에게 좋은 담보대출은 무엇인가요?",
    "저는 월 500만원 정도를 벌고 있습니다. 주택이 4억원이고 2억 정도를 대출받고 싶을 때 이에 적절한 상품이 있나요?, 상환 기간은 30년으로 생각하고 있어요",
    "그럼 월 상환액이 어느정도 되나요?"
]

for i, question in enumerate(questions, 1):
    result = chain({"question": question})
    print(f"\n질문 {i}: {question}")
    print(f"답변 {i}: {result['answer']}")
    print("-" * 50)



질문 1: 청년에게 좋은 담보대출은 무엇인가요?
답변 1: 청년에게 좋은 담보대출은 LTV 비율이 30%~70%인 대출 상품이 좋습니다. 또한 연 이자율이 0.8% ~ 1.5% 범위 내에 있는 대출 상품을 선택하는 것이 좋습니다. 이러한 조건을 충족하는 대출 상품을 찾아보시는 것이 좋습니다.
--------------------------------------------------

질문 2: 저는 월 500만원 정도를 벌고 있습니다. 주택이 4억원이고 2억 정도를 대출받고 싶을 때 이에 적절한 상품이 있나요?, 상환 기간은 30년으로 생각하고 있어요
답변 2: 주택 가격이 4억원이고 2억 정도를 대출 받고 싶을 때, 월 수입이 500만원인 경우에는 대출 한도인 LTV 30%~70%를 고려해야 합니다. 주어진 상황에서는 4억원 중 2억을 대출 받고자 하므로 LTV 50%에 해당합니다. 따라서 대출 한도 내에서 대출을 받을 수 있습니다. 

다만, 월 수입이 500만원이고 대출 상환액을 감당할 수 있는지에 대한 판단은 개인의 재정 상황과 부담 가능 여부에 따라 다를 수 있습니다. 대출 상품의 이자율과 상환 기간 등을 고려하여 월 상환액을 계산하고, 개인의 재정 상황을 ganz히 고려하여 결정하는 것이 중요합니다.
--------------------------------------------------

질문 3: 그럼 월 상환액이 어느정도 되나요?
답변 3: 주어진 컨텍스트에 따르면 월 상환액은 대출 원금에 대한 이자와 원금 상환액으로 구성됩니다. 이자율은 대출금리와 추가 이자율인 3%를 합산하여 계산됩니다. 대출 원금은 LTV 비율에 따라 30%에서 70% 사이로 결정되며, 대출 기간은 3년입니다. 따라서 월 상환액은 대출 원금과 이자를 합산하여 계산됩니다. 상환 시작일은 2024년 10월 21일이며, 상환 종료일은 지정되어 있지 않습니다. 최종 제출일은 2024년 10월 18일 11시입니다. 월 상환액을 정확히 계산하려면 대출 원금과 이자율을 고려하여 계

In [2]:
from langchain.document_loaders import JSONLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# # OpenAI API 키 설정
# import os
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# Step 1: 금융 상품 데이터를 JSON에서 로드하기
loader = JSONLoader(file_path="fss_test.json", jq_schema='.result.baseList[]', text_content=False)  # 금융 상품 데이터 JSON 파일
documents = loader.load()

# Step 2: 텍스트 분할기 사용 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
split_documents = text_splitter.split_documents(documents)

# Step 3: 임베딩 생성 및 벡터 스토어에 문서 저장
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_documents, embeddings)

# Step 4: OpenAI LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)  # 금융 추천에 적합한 톤과 안정성을 유지

# Step 5: RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# Step 6: 금융 상품 추천 기능 구현
def recommend_financial_product(user_query):
    # 사용자의 질의에 기반하여 추천 제공
    response = qa_chain.run(user_query)
    return response

# Step 7: 질의 예시
user_query = "저는 30대 직장인이고, 1억 정도의 대출을 하기 원합니다. 집은 3억 정도입니다."
recommendation = recommend_financial_product(user_query)
print(recommendation)


대출을 원하시는 금액이 1억 원이고, 집의 가치는 3억 원이라면, LTV(Loan To Value) 비율을 계산해보면 다음과 같습니다.

LTV = (대출금액 / 집의 가치) × 100 = (1억 / 3억) × 100 = 33.33%

이 경우 LTV 비율이 30%에서 70% 사이에 해당하므로 대출이 가능할 것으로 보입니다. 하지만 구체적인 대출 조건이나 금리는 금융기관에 따라 다를 수 있으니, 직접 상담을 받아보시는 것이 좋습니다.
